# This notebook adds a table of the number of checkins to busineeses on every hour at week

In [ ]:
import os
import sys
import numpy as np
import json
import matplotlib.pylab as plt
import pandas as pd
import getpass
%matplotlib inline 
sys.path.append("..") # go to parent dir
import unicodecsv as csv

Path Locations

In [2]:
username = getpass.getuser()
if username == 'sdekel':
    data_path = 'C:\DataSets\yelp_dataset\processed_data'
    checkin_path = 'C:\DataSets\yelp_dataset\dataset_restaurants'
elif username == 'zahi':
    data_path = 'D:\\users\\zahi\\OneDrive - Bar-Ilan University\\yelp_dataset_challenge_round9\\yelp2'
else:
    data_path = ''  ## TODO: PUT YOUR DATA PATH HERE

Dataset Names

In [3]:
business_data_name = 'yelp_academic_dataset_business_with_Clustered_location_with_Binary_Categories_clean_restaurants.csv'
business_fullpath = os.path.join( data_path, business_data_name )
checkin_data_name = 'yelp_academic_dataset_checkin_restaurants.json'
checkin_fullpath = os.path.join( checkin_path, checkin_data_name )

Read DataFrame

In [4]:
business_data = pd.read_csv(business_fullpath, encoding='latin1')
checkins= pd.read_json(checkin_fullpath, lines=True)
print('Number of columns = %d ' % len(business_data.columns))

Number of columns = 886 


Usefull functions

In [5]:
def chekinByDaysNP(ch):     #gets a checkin list, inputs numpy array
    days = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]
    ins = np.zeros(7 * 24)
    for i in ch:
        temp = i.split("-")
        day = temp[0]
        temp2 = temp[1].split(":")
        hour = temp2[0]
        num = temp2[1]
        ins[days.index(day) * 24 + int(hour)] = num
    return ins


def chekinByDaysList(ch):       #gets a checkin list, inputs list
    days = ["Sun", "Mon", "Tue", "Wed", "Thu", "Fri", "Sat"]
    ins = [0] * (7 * 24)
    for i in ch:
        temp = i.split("-")
        day = temp[0]
        temp2 = temp[1].split(":")
        hour = temp2[0]
        num = temp2[1]
        ins[days.index(day) * 24 + int(hour)] = num
    return ins


def chekinByDaysFrame(checkin):     #gets checkin dataframe, adds columns to every hour with num of checkins
    timeVector = checkin["time"].apply(lambda x: chekinByDaysList(x))
    times = pd.DataFrame()
    times = pd.DataFrame([t for t in timeVector])
    times.columns=["hour_"+str(value) for value in times.columns]
    return pd.concat([checkin, times], axis=1)

Transform checkins to binary data

In [6]:
checkin_binary = chekinByDaysFrame(checkins)

Merge business data and checkins

In [7]:
business_data_and_checkins = pd.merge(business_data, checkin_binary, on="business_id")
business_data_and_checkins = business_data_and_checkins.drop(axis=1,labels=["time","type"])

Show head

In [8]:
business_data_and_checkins.head()

,Unnamed: 0,attribute_AcceptsInsurance,attribute_AgesAllowed,attribute_Alcohol,attribute_Ambience_casual,attribute_Ambience_classy,attribute_Ambience_divey,attribute_Ambience_hipster,attribute_Ambience_intimate,attribute_Ambience_romantic,...,hour_158,hour_159,hour_160,hour_161,hour_162,hour_163,hour_164,hour_165,hour_166,hour_167
0,0,False,False,none,False,False,False,False,False,False,...,0,0,0,0,2,0,0,0,0,0
1,1,False,False,none,False,False,False,False,False,False,...,0,0,0,1,2,0,0,2,0,1
2,2,False,False,none,True,False,False,False,False,False,...,0,0,0,1,1,3,7,3,4,5
3,3,False,False,False,False,False,False,False,False,False,...,0,1,1,0,0,0,0,0,0,0
4,4,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0


In [9]:
print('Number of columns = %d ' % len(business_data_and_checkins.columns))

Number of columns = 1054 


Save to CSV:

In [10]:
business_data_name_to_save = 'yelp_academic_dataset_business_with_Clustered_location_with_Binary_Categories_with_checkins_clean_restaurants.csv'
business_fullpath = os.path.join( data_path, business_data_name_to_save )
business_data_and_checkins.to_csv(business_fullpath )